In [1]:
import os, glob, csv, warnings
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
from shapely.ops import unary_union
from fiona.env import Env
from pykrige.ok import OrdinaryKriging
from numpy.linalg import LinAlgError
from scipy.spatial import cKDTree

# -------- Paths
DATA_DIR   = r"C:\Users\krish\Desktop\SpatialCARE\DailyGPKG"
PASIG_SHP  = r"C:\Users\krish\Desktop\PhD Class\Shapefile\MM\Pasig\Pasig.shp"
OUT_F_DIR  = r"C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\kriging_local_contours"
OUT_CSV    = r"C:\Users\krish\Desktop\SpatialCARE\Outputs\exceedance_local_summary.csv"
os.makedirs(OUT_F_DIR, exist_ok=True)

# -------- Settings
WGS84, UTM51 = "+proj=longlat +datum=WGS84 +no_defs", "+proj=utm +zone=51 +datum=WGS84 +units=m +no_defs"
FIG_SIZE, FIG_DPI = (7,7), 150
GRID_RES_M = 100
VARIOGRAM_MODEL, RETRY_NUGGET = "spherical", 1e-3
COVERAGE_MAX_DIST_M = 3000

# Local guideline
BINS    = [0.0, 25.0, 35.0, 45.0, 55.0, 90.0, np.inf]
LABELS  = ["Good (0–25)","Fair (25.1–35)","Unhealthy (sens.) (35.1–45)",
           "Very unhealthy (45.1–55)","Acutely unhealthy (55.1–90)","Emergency (≥91)"]
COLORS  = ["#00E400","#FFFF00","#FF7E00","#FF0000","#8F3F97","#7E0023"]
CMAP_CAT = mpl.colors.ListedColormap(COLORS)
NORM_CAT = mpl.colors.BoundaryNorm(BINS, ncolors=len(COLORS), clip=False)

# Contours to label (pick a few to avoid clutter)
CONTOURS = [25, 35, 55, 90]

def pick_pm_col(cols):
    for c in cols:
        if str(c).lower() in ("pm25","pm_25","pm2_5","pm2.5"): return c
    return None

# Boundary & grid
with Env(SHAPE_RESTORE_SHX="YES"):
    pasig = gpd.read_file(PASIG_SHP)
if pasig.crs is None: pasig = pasig.set_crs(WGS84)
pasig_utm = pasig.to_crs(UTM51)
geom = unary_union(pasig_utm.geometry)
minx,miny,maxx,maxy = geom.bounds
xs = np.arange(minx, maxx+GRID_RES_M, GRID_RES_M)
ys = np.arange(miny, maxy+GRID_RES_M, GRID_RES_M)
grid_x, grid_y = np.meshgrid(xs, ys)

# CSV header
if not os.path.exists(OUT_CSV):
    with open(OUT_CSV, "w", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow([
            "date","stations",
            "pct_gt25","pct_gt35","pct_gt55","pct_ge91",
            "pct_good","pct_fair","pct_unsens","pct_very","pct_acute","pct_emerg",
            "total_area_km2"
        ])

files = sorted(glob.glob(os.path.join(DATA_DIR, "date_2025-*.gpkg")))
if not files: raise SystemExit("No daily GPKG files found.")

for f in files:
    base = os.path.splitext(os.path.basename(f))[0]
    day  = base.replace("date_","")

    g = gpd.read_file(f)
    if g.crs is None: g = g.set_crs(WGS84)
    if ("geometry" not in g) or g.geometry.is_empty.all():
        lon = next((c for c in g.columns if str(c).lower() in ("longitude","lon","x")), None)
        lat = next((c for c in g.columns if str(c).lower() in ("latitude","lat","y")), None)
        if lon and lat:
            g = g.set_geometry(gpd.points_from_xy(g[lon], g[lat], crs=g.crs))
        else:
            print("Skip (no geometry):", base); continue

    pm_col = pick_pm_col(g.columns)
    if pm_col is None: print("Skip (no PM):", base); continue

    pts = g.to_crs(UTM51)[[pm_col,"geometry"]].copy()
    pts["x"]=pts.geometry.x; pts["y"]=pts.geometry.y
    pts = pts.groupby(["x","y"], as_index=False)[pm_col].mean()
    x = pts["x"].values; y = pts["y"].values; z = np.clip(pts[pm_col].astype(float).values, 0, None)
    if len(z)<3: print("Skip (n<3):", base); continue

    # Kriging
    try:
        OK = OrdinaryKriging(x,y,z, variogram_model=VARIOGRAM_MODEL,
                             enable_plotting=False, verbose=False, coordinates_type="euclidean")
        zg, zv = OK.execute("grid", xs, ys, backend="loop"); zg = np.array(zg)
    except LinAlgError:
        OK = OrdinaryKriging(x,y,z, variogram_model=VARIOGRAM_MODEL, nugget=RETRY_NUGGET,
                             enable_plotting=False, verbose=False, coordinates_type="euclidean")
        zg, zv = OK.execute("grid", xs, ys, backend="loop"); zg = np.array(zg)

    # Masks
    tree = cKDTree(np.c_[x,y]); dists,_ = tree.query(np.c_[grid_x.ravel(), grid_y.ravel()], k=1)
    cover = dists.reshape(grid_x.shape) <= COVERAGE_MAX_DIST_M
    centers = gpd.GeoSeries(gpd.points_from_xy(grid_x.ravel(), grid_y.ravel()), crs=UTM51)
    inside  = centers.within(geom).values.reshape(zg.shape)
    zmask = np.where(cover & inside, np.maximum(zg,0.0), np.nan)

    if not np.isfinite(zmask).any():
        print("Skip (no valid cells):", base); continue

    # Percent area by thresholds
    cell_area_km2 = (GRID_RES_M*GRID_RES_M)/1e6
    total_area_km2 = np.isfinite(zmask).sum()*cell_area_km2
    pct_gt25 = 100.0*(np.nan_to_num(zmask>25).sum()*cell_area_km2)/max(1e-9,total_area_km2)
    pct_gt35 = 100.0*(np.nan_to_num(zmask>35).sum()*cell_area_km2)/max(1e-9,total_area_km2)
    pct_gt55 = 100.0*(np.nan_to_num(zmask>55).sum()*cell_area_km2)/max(1e-9,total_area_km2)
    pct_ge91 = 100.0*(np.nan_to_num(zmask>=91).sum()*cell_area_km2)/max(1e-9,total_area_km2)

    # Percent by categories
    idx = np.digitize(zmask, BINS, right=True) - 1  # -1..5
    cats = [(idx==i).sum() for i in range(6)]
    pct_by_cat = [100.0*(c*cell_area_km2)/max(1e-9,total_area_km2) for c in cats]

    # Plot categorical with local contours
    cat_mask = np.ma.masked_where(~np.isfinite(zmask), idx)
    fig, ax = plt.subplots(figsize=FIG_SIZE, dpi=FIG_DPI)
    im = ax.imshow(cat_mask, extent=[xs[0], xs[-1], ys[0], ys[-1]],
                   origin="lower", cmap=CMAP_CAT, norm=NORM_CAT)
    pasig_utm.boundary.plot(ax=ax, color="black", linewidth=0.8)

    try:
        cs = ax.contour(grid_x, grid_y, zmask, levels=CONTOURS,
                        colors="black", linewidths=0.9, linestyles=["--","-","-","-"])
        ax.clabel(cs, fmt=lambda v: f"{int(v)} µg/m³", inline=True, fontsize=8)
    except Exception:
        pass

    subtitle = f"%>25: {pct_gt25:.1f} • %>35: {pct_gt35:.1f} • %>55: {pct_gt55:.1f} • %≥91: {pct_ge91:.1f} • Stations: {len(z)}"
    ax.set_title(f"PM₂.₅ (Local guideline) • {day}\n{subtitle}")
    ax.set_axis_off()
    patches = [mpatches.Patch(color=c, label=l) for c,l in zip(COLORS, LABELS)]
    ax.legend(handles=patches, loc="upper left", bbox_to_anchor=(1.01, 1.0), fontsize=9, frameon=True)

    out = os.path.join(OUT_F_DIR, f"{base}_kriged_localContours.png")
    plt.tight_layout(); plt.savefig(out, bbox_inches="tight"); plt.close(fig)
    print("Saved:", out)

    # CSV row
    with open(OUT_CSV, "a", newline="", encoding="utf-8") as fcsv:
        csv.writer(fcsv).writerow([
            day, len(z),
            pct_gt25, pct_gt35, pct_gt55, pct_ge91,
            *pct_by_cat, total_area_km2
        ])


Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\kriging_local_contours\date_2025-02-06_kriged_localContours.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\kriging_local_contours\date_2025-02-07_kriged_localContours.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\kriging_local_contours\date_2025-02-08_kriged_localContours.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\kriging_local_contours\date_2025-02-09_kriged_localContours.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\kriging_local_contours\date_2025-02-10_kriged_localContours.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\kriging_local_contours\date_2025-02-11_kriged_localContours.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\kriging_local_contours\date_2025-02-12_kriged_localContours.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\kriging_local_contours\date_2025-02-13_kriged_localContours.png
Saved: C:\Users\krish\De